# Weekly project

Today you are going to implement the last parts of the algorithm you started on monday. For reference you can see it below.

![title](algorithm_3.png)

It is a good idea to follow and track the steps in the algorithm in the below implementation. Only take one step at a time.

Once you have the algorithm up and running you can try with a larger dataset to see if your algorithm is able to maintain good accurracy over a longer distance. The larger dataset can be found here:
[Left images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_win_dtu_dk/EQu8kmGBDDROtGJ7IkZB2tQBJrxmgY9t8LVM_JuEi83TYw?e=GiOby4)
[Right images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_win_dtu_dk/EcKI_zrXTvpMulizidCZm4oBLJcQ_LTV9Zs6oQFF74JTRQ?e=6bgQVw)

In [3]:
import numpy as np
import cv2 as cv2
from numpy.linalg import inv, pinv
import matplotlib.pyplot as plt
import time as t
from helpers import *


def extract_keypoints_surf(img1, img2, K, baseline):
    """
    use surf to detect keypoint features
    remember to include a Lowes ratio test
    """
    # gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    # gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    surf_descriptor = cv2.xfeatures2d.SURF_create()
    kp1, des1 = surf_descriptor.detectAndCompute(img1, None) #Keypoints and descriptions
    kp2, des2 = surf_descriptor.detectAndCompute(img2, None)

    matcher = cv2.DescriptorMatcher_create("BruteForce")
    rawMatches = matcher.knnMatch(des1, des2, 2) # top two matches
    matches = []
    ratio = 0.75

    # loop over the raw matches
    for m in rawMatches:
        # ensure the distance is within a certain ratio of each other (i.e. Lowe's ratio test)
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
            matches.append((m[0].trainIdx, m[0].queryIdx))    

    match_points1 = np.float32([kp1[i].pt for (_, i) in matches])
    match_points2 = np.float32([kp2[i].pt for (i, _) in matches])

    p1 = np.array(match_points1)
    p2 = np.array(match_points2)

    ##### ############# ##########
    ##### Do Triangulation #######
    ##### ########################
    #project the feature points to 3D with triangulation
    
    #projection matrix for Left and Right Image
    M_left = K.dot(np.hstack((np.eye(3), np.zeros((3, 1)))))
    M_rght = K.dot(np.hstack((np.eye(3), np.array([[-baseline, 0, 0]]).T)))

    # homogeneous coordinates
    p1_flip = np.vstack((p1.T, np.ones((1, p1.shape[0]))))
    p2_flip = np.vstack((p2.T, np.ones((1, p2.shape[0]))))

    P = cv2.triangulatePoints(M_left, M_rght, p1_flip[:2], p2_flip[:2])

    # Normalize homogeneous coordinates (P->Nx4  [N,4] is the normalizer/scale)
    P = P / P[3]
    land_points = P[:3]

    return land_points.T, p1


def featureTracking(img_1, img_2, p1, world_points):
    """
    Use OpenCV to find the prev_points from the prev_img in the next_img
    Remember to remove points that could not be found from prev_points, next_points, and world_points
    hint: status == 1
    """
    params = dict(winSize=(21, 21),
                 maxLevel=3,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    # the p1 must be float32, not float64
    p2, status, _ = cv2.calcOpticalFlowPyrLK(img_1, img_2, p1, None, **params)

    index= np.array([ i==1 for i in status ])
    #print((index == False).any())
    
    p1 = p1[index.flatten()]
    p2 = p2[index.flatten()]
    world_points = world_points[index.flatten()]

    return world_points, p1, p2

def playImageSequence(left_img, right_img, K):

    baseline = 0.54
    ##### ################################# #######
    ##### Get 3D points Using Triangulation #######
    ##### #########################################

    """
    Implement step 1.2 and 1.3
    Store the features in 'reference_2D' and the 3D points (landmarks) in 'landmark_3D'
    hint: use 'extract_keypoints_surf' above
    """
    landmark_3D, reference_2D = extract_keypoints_surf(left_img, right_img, K, baseline)

    # reference
    reference_img = left_img

    # Groundtruth for plot
    truePose = getTruePose()
    traj = np.zeros((600, 600, 3), dtype=np.uint8)
    maxError = 0

    for i in range(1, 100):
        print('image: ', i)
        curImage = getLeftImage(i)
        curImage_R = getRightImage(i)

        ##### ############################################################# #######
        ##### Calculate 2D and 3D feature correspndances in t=T-1, and t=T  #######
        ##### #####################################################################
        """
        Implement step 2.2)
        Remember this is a part of a loop, so the initial features are already
        provided in step 1)-1.3) outside the loop in 'reference_2D' and 'landmark_3D'
        """
        landmark_3D, reference_2D, tracked_2Dpoints = featureTracking(reference_img, 
                                                                      curImage, 
                                                                      reference_2D,
                                                                      landmark_3D)
        ##### ################################# #######
        ##### Calculate relative pose using PNP #######
        ##### #########################################
        """
        Implement step 2.3)
        """
        _, rvec, tvec, inliers = cv2.solvePnPRansac(landmark_3D,tracked_2Dpoints,K, None)

        ##### ####################################################### #######
        ##### Get Pose and Tranformation Matrix in world coordionates #######
        ##### ###############################################################
        rot, _ = cv2.Rodrigues(rvec)

        # coordinate transformation, from camera to world. What is the XYZ of the camera wrt World
        tvec = -rot.T.dot(tvec)
        # inverse transform. A tranform projecting points from the camera frame to the world frame 
        inv_transform = np.hstack((rot.T, tvec))  

        ##### ################################# #######
        ##### Get 3D points Using Triangulation #######
        ##### #########################################
        # re-obtain the 3D points
        """
        Implement step 2.4)
        """
        # it is quite slow if we re-extract features every time
        landmark_3D_new, reference_2D_new = extract_keypoints_surf(curImage, curImage_R, K, baseline)

        #Project the points from camera to world coordinates
        reference_2D = reference_2D_new.astype('float32')

        landmark_3D = inv_transform @ np.vstack((landmark_3D_new.T, np.ones((1, landmark_3D_new.shape[0]))))
        landmark_3D = landmark_3D.T

        ##### ####################### #######
        ##### Done, Next image please #######
        ##### ###############################
        reference_img = curImage

        ##### ################################## #######
        ##### START OF Print and visualize stuff #######
        ##### ##########################################
        # draw images
        draw_x, draw_y = int(tvec[0]) + 300, 600-(int(tvec[2]) + 100)
        true_x, true_y = int(truePose[i][3]) + 300, 600-(int(truePose[i][11]) + 100)

        curError = np.sqrt(
            (tvec[0] - truePose[i][3]) ** 2 +
            (tvec[1] - truePose[i][7]) ** 2 +
            (tvec[2] - truePose[i][11]) ** 2)
        
        if (curError > maxError):
            maxError = curError

        print(tvec[0],tvec[1],tvec[2], rvec[0], rvec[1], rvec[2])
        print([truePose[i][3], truePose[i][7], truePose[i][11]])
        
        text = "Coordinates: x ={0:02f}m y = {1:02f}m z = {2:02f}m".format(float(tvec[0]), float(tvec[1]),
                                                                           float(tvec[2]))
        cv2.circle(traj, (draw_x, draw_y), 1, (0, 0, 255), 2)
        cv2.circle(traj, (true_x, true_y), 1, (255, 0, 0), 2)
        cv2.rectangle(traj, (10, 30), (550, 50), (0, 0, 0), cv2.FILLED)
        cv2.putText(traj, text, (10, 50), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, 8)

        h1, w1 = traj.shape[:2]
        h2, w2 = curImage.shape[:2]
        vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)
        vis[:h1, :w1, :3] = traj
        vis[:h2, w1:w1 + w2, :3] = np.dstack((np.dstack((curImage,curImage)),curImage))

        cv2.imshow("Trajectory", vis)
        k = cv2.waitKey(1) & 0xFF
        if k == 27: break


    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print('Maximum Error: ', maxError)
    ##### ################################ #######
    ##### END OF Print and visualize stuff #######
    ##### ########################################

if __name__ == '__main__':
    
    left_img = getLeftImage(0)
    right_img = getRightImage(0)

    K = getK()

    playImageSequence(left_img, right_img, K)


image:  1
[-0.00142721] [-0.00746259] [0.67569242] [-0.00216076] [0.00330786] [-0.00259909]
[-0.04690294, -0.02839928, 0.8586941]
image:  2
[-0.00832788] [-0.01208252] [1.3725927] [-0.00380039] [0.00749377] [-0.00137162]
[-0.09374345, -0.05676064, 1.716275]
image:  3
[-0.02826311] [-0.01851176] [2.09259195] [-0.00542558] [0.01118823] [-0.00118673]
[-0.1406429, -0.08515762, 2.574964]
image:  4
[-0.04229995] [-0.02462947] [2.8285532] [-0.00630466] [0.01637761] [0.0002034]
[-0.1874858, -0.1135202, 3.432648]
image:  5
[-0.06180695] [-0.03747599] [3.5776563] [-0.00658506] [0.02067266] [-0.00079083]
[-0.2343818, -0.141915, 4.291335]
image:  6
[-0.08697672] [-0.04670832] [4.33798545] [-0.0081169] [0.02518499] [0.00118055]
[-0.2812195, -0.1702743, 5.148987]
image:  7
[-0.12049588] [-0.05742262] [5.1108103] [-0.00896588] [0.03009992] [0.00397711]
[-0.3281178, -0.1986703, 6.007777]
Maximum Error:  [0.93145426]


# Challenge 
The current implementation only uses features computed at the current timestep. However, as we process more images we potentially have a lot of features from previous timesteps that are still valid. The challenge is to expand the `extract_keypoints_surf(..., refPoints)` function by giving it old reference points. You should then combine your freshly computed features with the old features and remove all duplicates. This requires you to keep track of old features and 3D points.

Hint 1: look in `helpers.py` for removing duplicates.

Hint 2: you are not interested in points that are behind you, so remember to remove points that are negative in the direction you move.